In [292]:
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings('ignore')

In [300]:
import mysql.connector
import pandas as pd


connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="hymnastic",
    auth_plugin='mysql_native_password'
)


query = "SELECT * FROM songs"
songs_df = pd.read_sql(query, con=connection)

# Close the connection
connection.close()


songs_df

,sno,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73.0,230666,0,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic\r
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55.0,149610,0,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic\r
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57.0,210826,0,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic\r
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71.0,201933,0,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic\r
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82.0,198853,0,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21.0,384999,0,0.172,0.2350,...,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music\r
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22.0,385000,0,0.174,0.1170,...,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music\r
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22.0,271466,0,0.629,0.3290,...,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music\r
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41.0,283893,0,0.587,0.5060,...,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music\r


In [3]:
songs_df.columns

Index(['Sno', 'track_id', 'artists', 'album_name', 'track_name', 'popularity',
       'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature', 'track_genre'],
      dtype='object')

In [199]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Assuming you have a DataFrame named 'df' with the necessary columns
columns_to_use = ['popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

# Standardize Data
scaler = StandardScaler()

# Recommendation Function
def recommend_song(song_name, artists, df, num_recommendations=5):
    # Filter the DataFrame based on the song and artist
    sub_df = df[(df['track_name'] == song_name) & (df['artists'] == artists)]

    if sub_df.empty:
        return "Song not found in the dataset."

    # Extract all unique genres for the given song
    song_genre = set(sub_df['track_genre'])

    # Create a sub-DataFrame with all genres for the given song
    sub_df_all_genres = df[(df['track_genre']).isin(song_genre)]

    # Check the number of samples in the subset
    num_samples = len(sub_df_all_genres)
    
    # Adjust n_neighbors based on the number of samples
    k = min(num_recommendations, num_samples)  # Set to the smaller of specified recommendations or the number of samples

    # Standardize the features for the subset
    df_subset_scaled = scaler.fit_transform(sub_df_all_genres[columns_to_use])

    # Fit KNN Model on the subset
    knn_model = NearestNeighbors(n_neighbors=num_samples, algorithm='auto')
    knn_model.fit(df_subset_scaled)

    # Use the features of the specified song for recommendation
    song_features_scaled = scaler.transform(sub_df[columns_to_use])
    _, indices = knn_model.kneighbors(song_features_scaled)

    # Get the recommended songs based on indices
    recommended_songs = sub_df_all_genres.iloc[indices[0]][['track_name', 'artists', 'track_genre']]

    # Exclude the original song from the recommendations
    recommended_songs = recommended_songs[recommended_songs['track_name'] != song_name]

    # Ensure a fixed number of recommendations
    recommended_songs = recommended_songs.head(num_recommendations)

    # Filter out duplicate track names and artists
    recommended_songs = recommended_songs.drop_duplicates(subset=['track_name', 'artists'])

    # If the number of recommendations is less than the specified, fill with additional recommendations
    if len(recommended_songs) < num_recommendations:
        remaining_recommendations = df[df['track_name'] != song_name].sample(n=num_recommendations - len(recommended_songs))
        recommended_songs = pd.concat([recommended_songs, remaining_recommendations])

    return recommended_songs.head(num_recommendations)



In [201]:
# Example usage
song_name_to_search = 'HIGHEST IN THE ROOM'
artist_name_to_search ='Travis Scott'

recommendations = recommend_song(song_name_to_search, artist_name_to_search, songs_df)
recommendations

,track_name,artists,track_genre
51007,All Night (Live),AP Dhillon;Shinda Kahlon,hip-hop
51071,Final Thoughts,AP Dhillon;Shinda Kahlon;Gminxr,hip-hop
53473,Hurts So Good - Slow + Reverb,Astrid S,house
51778,Sagiyae,Stephen Zechariah,hip-hop
53200,There for You,Martin Garrix;Troye Sivan,house


In [9]:
import mysql.connector
import pandas as pd


connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="hymnastic",
    auth_plugin='mysql_native_password'
)


query = "SELECT * FROM lyric"
lyric_df = pd.read_sql(query, con=connection)

# Close the connection
connection.close()


lyric_df 

C:\Users\aashi\AppData\Local\Temp\ipykernel_15196\1675467163.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  lyric_df = pd.read_sql(query, con=connection)


,artist,title,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assuming you already have a DataFrame named lyric_df
# with columns: artist, song, link, text

def recommend_songs(input_lyrics, lyric_df, num_recommendations=5):
    # Create a new DataFrame with the input lyrics
    new_row = pd.DataFrame({'artist': ['Input'], 'song': ['Input Song'], 'link': [''], 'text': [input_lyrics]})
    lyric_df = pd.concat([lyric_df, new_row], ignore_index=True)

    # Use TfidfVectorizer to convert text data into numerical features
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(lyric_df['text'])

    # Calculate cosine similarity between input lyrics and all other lyrics
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

    # Get the indices of the top N most similar songs
    recommended_song_indices = cosine_similarities.argsort()[0][-num_recommendations:][::-1]

    # Get the details of the recommended songs
    recommended_songs = lyric_df.iloc[recommended_song_indices]

    # Return the recommended songs details as a list of tuples
    recommendations = [(row['artist'], row['title']) for _, row in recommended_songs.iterrows()]
    return recommendations

# Example usage:
input_lyrics = "nigga"
# Assuming lyric_df is your DataFrame

# Call the recommendation function
recommendations = recommend_songs(input_lyrics, lyric_df)

# Print the recommended songs
for i, (artist, song) in enumerate(recommendations, 1):
    print(f"{i}. Recommended Song: {song} by {artist}")


1. Recommended Song: What It Do by Yukmouth
2. Recommended Song: We Made It Freestyle by Drake
3. Recommended Song: Whatchu Want by Notorious B.I.G.
4. Recommended Song: Who Do You Love by YG
5. Recommended Song: Them Braves by Ying Yang Twins


In [29]:
import mysql.connector
import pandas as pd


connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="hymnastic",
    auth_plugin='mysql_native_password'
)


query = "SELECT * FROM songs"
songs_df = pd.read_sql(query, con=connection)

# Close the connection
connection.close()


songs_df

C:\Users\aashi\AppData\Local\Temp\ipykernel_14452\2475634713.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  songs_df = pd.read_sql(query, con=connection)


,sno,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73.0,230666,0,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic\r
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55.0,149610,0,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic\r
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57.0,210826,0,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic\r
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71.0,201933,0,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic\r
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82.0,198853,0,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21.0,384999,0,0.172,0.2350,...,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music\r
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22.0,385000,0,0.174,0.1170,...,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music\r
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22.0,271466,0,0.629,0.3290,...,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music\r
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41.0,283893,0,0.587,0.5060,...,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music\r


In [30]:
songs_df.columns

Index(['sno', 'track_id', 'artists', 'album_name', 'track_name', 'popularity',
       'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature', 'track_genre'],
      dtype='object')

In [31]:
columns_to_drop = ['sno', 'track_id', 'album_name', 'track_name', 'key', 'mode','duration_ms','explicit','time_signature']

# Drop the specified columns
artists_df = songs_df.drop(columns=columns_to_drop, axis=1)

In [33]:
artists_df

,artist
0,Gen Hoshino
1,Ben Woodward
2,Kina Grannis
3,Chord Overstreet
4,Tyrone Wells
...,...
15033,Estudio Hermoso
15034,Christy Nockels
15035,Paz Interior
15036,Cuencos Tibetanos Sonidos Relajantes


In [176]:
import pandas as pd

# Group by 'artists', 'track_genre', and calculate the mean for numeric columns
numeric_columns = artists_df.select_dtypes(include=['number']).columns
average_df = artists_df.groupby(['artists', 'track_genre'], as_index=False)[numeric_columns].mean()

# Optionally, you can add the count of each genre for an artist
genre_counts = artists_df.groupby(['artists', 'track_genre'], as_index=False).size().rename(columns={'size': 'genre_count'})

# Merge the two DataFrames on 'artists' and 'track_genre'
result_df = pd.merge(average_df, genre_counts, on=['artists', 'track_genre'])


In [255]:
result_df[result_df['artists']=='Justin Bieber']

,artists,track_genre,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre_count
18630,Justin Bieber,pop,5.103448,0.597621,0.662638,-7.059586,0.067857,0.394307,0.000135,0.143859,0.684948,146.617034,58


In [286]:
from sklearn.metrics.pairwise import cosine_similarity

def get_artist_recommendations(artist_name, result_df, num_recommendations=5):
    # Filter the DataFrame to include only the rows for the given artist
    artist_data = result_df[result_df['artists'] == artist_name]

    if artist_data.empty:
        print(f"Artist '{artist_name}' not found.")
        return []

    # Extract numeric columns for similarity calculation
    numeric_data = artist_data[numeric_columns]
    
    # Extract the genres of the input artist
    input_genres = artist_data['track_genre'].tolist()

    # Filter the result_df to include only artists from the same genres
    sub_df = result_df[(result_df['track_genre']).isin(input_genres)]

    if sub_df.empty:
        print(f"No similar artists found in the same genre for '{artist_name}'.")
        return []

    # Calculate cosine similarity with other artists
    similarities = cosine_similarity(numeric_data, sub_df[numeric_columns])

    # Get indices of artists with the highest similarity
    similar_artist_indices = similarities.argsort()[0][::-1][1:]

    # Get recommended artists (limited to 5)
    recommended_artists = sub_df.iloc[similar_artist_indices[:num_recommendations]]['artists'].tolist()

    return recommended_artists




In [291]:
# Example: Get recommendations for a given artist (limited to 3)
user_input_artist = "Justin Bieber"
recommendations = get_artist_recommendations(user_input_artist, result_df, num_recommendations=3)

# Check if recommendations are available before printing
if recommendations:
    # Print only the first 5 recommended artists, stopping after the third semicolon
    for artist in recommendations:
        artists = artist.split(";")
        for i, individual_artist in enumerate(artists):
            if i < 2:
                print(individual_artist.strip())  # Add strip() to remove leading/trailing spaces
            else:
                break
   


Lana Del Rey
Cedric Gervais
David Guetta
Dua Lipa
Saweetie
